In [1]:
## create a polygon in tkinter

import tkinter as tk

# start the main tkinter window
window = tk.Tk()
# create a canvas
canvas = tk.Canvas(window, background='white')
# create a polygon
canvas.create_polygon(150, 75, 225, 30, 245, 75, 225, 150, 30, 46, outline='gray', fill='gray', width=2)
# display the canvas
canvas.pack()
# run the application
window.mainloop()

In [ ]:
## pyproj

from pyproj import Proj
mercator = Proj(init='epsg:3395')

# geographic coordinates of Rimini
# Longitude: 12.5652800°
# Latitude: 44.0575500°
longitude, latitude = 12.56528, 44.05755

# convert geographic coordinates to projected coordinates
rimini_x, rimini_y = mercator(longitude, latitude)

# convert projected coordinates to geographic coordinates
rimini_lon, rimini_lat = mercator(rimini_x, rimini_y, inverse=True)

print('projected coordinates: x={}, y={}'.format(rimini_x, rimini_y))
print('geographic coordinates: lon={}, lat={}'.format(rimini_lon, rimini_lat))

In [ ]:
## extract the polygons of a shapefile 

import shapely.geometry
from shapefile import Reader

path_to_shapefile = 'C:/Users/minto/Desktop/EuroPython 2017/pyGISS - Jupyter notebook/Continents.shp'

# open the .shp file with the shapefile Reader
sf = Reader(path_to_shapefile)  
# retrieves the shapefile's shapes
shapes = sf.shapes() 
for obj in shapes:
    # convert shapefile geometries into shapely geometries
    shape = shapely.geometry.shape(obj)
    # - geom_type tells us if it is a polygon or a multipolygon
    # - the length of a multipolygon tells us how many polygons it contains
    # - a shapely multipolygon is an iterable
    print(shape.geom_type, len(shape), hasattr(shape, '__iter__'), shape[0].geom_type)


In [4]:
import tkinter as tk
import shapely.geometry
import shapefile
import pyproj

window = tk.Tk()
canvas = tk.Canvas(window, width=800, height=600, background='white')
# to be replaced with local path to a shapefile
path_to_shapefile = 'C:/Users/minto/Desktop/EuroPython 2017/pyGISS - Jupyter notebook/World countries.shp'
        
def to_projected_coordinates(longitude, latitude):
    px, py = pyproj.Proj(init='epsg:3395')(longitude, latitude)
    # in tkinter, the y axis is reversed, we take the inverse of the projected y coordinate
    return px, -py

sf = shapefile.Reader(path_to_shapefile)       
polygons = sf.shapes() 
for polygon in polygons:
    # convert shapefile geometries into shapely geometries
    # to extract the polygons of a multipolygon
    polygon = shapely.geometry.shape(polygon)
    # if it is a polygon, we use a list to make it iterable
    if polygon.geom_type == 'Polygon':
        polygon = [polygon]
    for land in polygon:
        land = str(land)[10:-2].replace(', ', ',').replace(' ', ',')
        coords = land.replace('(', '').replace(')', '').split(',')
        canvas.create_polygon(sum((to_projected_coordinates(*c) for c in zip(coords[0::2], coords[1::2])), ()))
    
# windows zoom
canvas.bind('<MouseWheel>', lambda e: canvas.scale('all', e.x, e.y, (e.delta>0)*1.2 or 0.8, (e.delta>0)*1.2 or 0.8))
# unix zoom
canvas.bind('<Button-4>', lambda e: canvas.scale('all', e.x, e.y, 1.2, 1.2))
canvas.bind('<Button-5>', lambda e: canvas.scale('all', e.x, e.y, 0.8, 0.8))
canvas.pack()
window.mainloop()

In [6]:
from tkinter import filedialog
import tkinter as tk
import shapely.geometry
import shapefile
import pyproj

window = tk.Tk()
canvas = tk.Canvas(window, width=800, height=600, background='white')
        
def to_projected_coordinates(longitude, latitude):
    px, py = pyproj.Proj('+proj=ortho +lon_0=28 +lat_0=47')(longitude, latitude)
    # in tkinter, the y axis is reversed, we take the inverse of the projected y coordinate
    return px, -py

# ask the user to choose a file
filepath ,= tk.filedialog.askopenfilenames(title='Import shapefile')
sf = shapefile.Reader(filepath)       
polygons = sf.shapes() 
for polygon in polygons:
    # convert shapefile geometries into shapely geometries
    # to extract the polygons of a multipolygon
    polygon = shapely.geometry.shape(polygon)
    # if it is a polygon, we use a list to make it iterable
    if polygon.geom_type == 'Polygon':
        polygon = [polygon]
    for land in polygon:
        land = str(land)[10:-2].replace(', ', ',').replace(' ', ',')
        coords = land.replace('(', '').replace(')', '').split(',')
        canvas.create_polygon(sum((to_projected_coordinates(*c) for c in zip(coords[0::2], coords[1::2])), ()))
    
# windows zoom
canvas.bind('<MouseWheel>', lambda e: canvas.scale('all', e.x, e.y, (e.delta>0)*1.2 or 0.8, (e.delta>0)*1.2 or 0.8))
# unix zoom
canvas.bind('<Button-4>', lambda e: canvas.scale('all', e.x, e.y, 1.2, 1.2))
canvas.bind('<Button-5>', lambda e: canvas.scale('all', e.x, e.y, 0.8, 0.8))
# canvas sliding with right-click button
canvas.bind('<ButtonPress-3>', lambda e: canvas.scan_mark(e.x, e.y))
canvas.bind('<B3-Motion>', lambda e: canvas.scan_dragto(e.x, e.y, gain=1))
canvas.pack()
window.mainloop()